In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
import pyreadstat
from unidecode import unidecode

In [4]:
import pandas as pd
import pytask

from project_mp.config import BLD, SRC
from project_mp.data_management.process_distance_data import (
    process_distance_data,
)

def task_process_distance_data(
        data_path = SRC / "data" / "Mission_coordinatesok.csv",
        produces = BLD / "data" / "distance_to_missions.pickle",
):
    """Clean the distance data set."""
    process_distance_data(data_path, produces)
    processed_data = pd.read_csv(produces, encoding='latin1')
    processed_data.to_pickle(produces)

#apply the function to the data
task_process_distance_data()

OSError: Cannot save file into a non-existent directory: '/Users/moisespedrozo/final-project-cmpedrozopy/bld/data'

In [3]:
#Write a function to do the same that I did with the distance_df data set
def process_distance_data(input_file, output_file):
    """
    Load a CSV file, rename a column for easier merging, and save the processed data as a new CSV file.
    
    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the output CSV file.
    """
    distance_df = pd.read_csv(input_file, encoding='latin1')
    distance_df.rename(columns={'country':'mission_country'}, inplace=True)
    distance_df.to_csv(output_file, index=False)

#Apply the function to "C:\Users\HP\Desktop\Felipe\Moises\Distance\Mission_coordinatesok.csv" and save the output to "C:\Users\HP\Desktop\Felipe\Moises\Distance\epp\distance_to_missions_epp.csv"
#process_distance_data("C:/Users/HP/Desktop/Felipe/Moises/Distance/Mission_coordinatesok.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")
process_distance_data("/Users/moisespedrozo/final-project-cmpedrozopy/src/project_mp/data/Mission_coordinatesok.csv", "/Users/moisespedrozo/Bonn/epp/distance_to_missions_epp.csv")

In [5]:
# Define the function to process shapefiles
def shape_to_csv(department_path):
    # Read and reproject the shp files
    shape = gpd.read_file(department_path)
    shape = shape.to_crs(epsg=4326)
    # Calculate the centroid and extract the longitude and latitude
    shape['centroid'] = shape.centroid
    shape['longitude'] = shape['centroid'].x
    shape['latitude'] = shape['centroid'].y
    # Save as CSV in the same folder as the shapefile for easier access
    output_csv = department_path.replace(".shp", ".csv")
    shape.to_csv(output_csv, index=False)

# Root directory containing the folders and shapefiles
root_dir = "Users/moisespedrozo/Bonn/epp/shape_files"

# Go through all subdirectories and apply the function
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".shp") and file.startswith("Distritos_"):
            department_path = os.path.join(root, file) 
            shape_to_csv(department_path)


In [ ]:
from project_mp.config import BLD, SRC
from project_mp.data_management.shape_to_csv import shape_to_csv

def task_process_shapefiles(
        root_dir=SRC / "shape_files",
        produces=[BLD / "data" / f"{file.replace('.shp', '.csv')}" for file in os.listdir(SRC / "shape_files") if file.endswith(".shp") and file.startswith("Distritos_")]
):
    """Convert shapefiles to CSV format, extracting centroid coordinates."""
    for file in os.listdir(root_dir):
        if file.endswith(".shp") and file.startswith("Distritos_"):
            department_path = root_dir / file
            shape_to_csv(department_path)


In [166]:
def append_csvs_to_one(output_file, root_dir):
    # List to store individual DataFrames
    all_data = []

    # Walk through all subdirectories to find CSV files
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(root, file)
                print(f"Appending: {file_path}")
                # Read the CSV and append to the list
                df = pd.read_csv(file_path)
                all_data.append(df)

    # Concatenate all DataFrames into one
    deparmentos_shape_ok = pd.concat(all_data, ignore_index=True)
    # Delete duplicates in CLAVE variable
    deparmentos_shape_ok = deparmentos_shape_ok.drop_duplicates(subset=['CLAVE'])

    # Save the combined DataFrame as a single CSV
    deparmentos_shape_ok.to_csv(output_file, index=False)
    print(f"All CSV files appended and saved to: {output_file}")

#Apply the function
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"
output_file = "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv"

append_csvs_to_one(output_file, root_dir)

Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira_shp_data.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\alto_parana\Distritos_Alto_Parana.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\caazapa\Distritos_Caazapa.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira\Distritos_Guaira.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\itapua\Distritos_Itapua.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\misiones\Distritos_Misiones.csv
All CSV files appended and saved to: C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv


In [167]:

# Function to compute the closest mission for each department
def find_closest_location(distritos, missions):
    # Normalize longitude and latitude, except for rows where DPTO_DESC == 'GUAIRÁ'
    

    results = []

    for index, dep in distritos.iterrows():
        # Skip if department coordinates are invalid
        if pd.isna(dep['latitude']) or pd.isna(dep['longitude']):
            continue

        dep_coords = (dep['latitude'], dep['longitude'])
        
        # Calculate distances to all missions
        missions['distance'] = missions.apply(
            lambda x: geodesic(dep_coords, (x['latitude'], x['longitude'])).kilometers, axis=1
        )
        
        # Find the closest mission and keep the DPTO_DESC from distritos
        closest_mission = missions.loc[missions['distance'].idxmin()]
        results.append({
            'department_index': index,
            'department_code': dep.get('DPTO', 'Unknown'),
            'department_name': dep.get('DPTO_DESC', 'Unknown'),
            'distrito': dep.get('DIST_DESC', 'Unknown'),
            'mission_name': closest_mission['mission_name'],
            'mission_latitude': closest_mission['latitude'],
            'mission_longitude': closest_mission['longitude'],
            'distance_to_mission_km': closest_mission['distance']
        })
    
    return pd.DataFrame(results)

# Example usage
distritos = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv")
missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

# Compute the closest missions
closest_missions_df = find_closest_location(distritos, missions)

# Save the results
closest_missions_df.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv", index=False)

print("Closest missions have been calculated and saved!")

#Now, make a function so that DIST_DESC observations are in lowercase and clean the name of the cities such that "ascii" characters are removed
def clean_city_names(input_file, output_file):
    # Load the data
    closest_missions = pd.read_csv(input_file, encoding='latin1')
    
    # Lowercase the DIST_DESC column
    closest_missions['distrito'] = closest_missions['distrito'].str.lower()
    # Remove accents and non-ASCII characters
    closest_missions['distrito'] = closest_missions['distrito'].apply(lambda x: unidecode(x))
    # Later, I will merge this data set with the survey data. So, I will use a key with the cleaned city name + department code
    # Thus, If departamento has only one digit, add a leading zero. Also, make department_code string first
    #closest_missions['department_code'] = closest_missions['department_code'].astype(str)
    #closest_missions['department_code'] = closest_missions['department_code'].apply(lambda x: f"0{x}" if len(str(x)) == 1 else x)

    # Save the cleaned data
    closest_missions.to_csv(output_file, index=False, encoding='latin1')
    # Remove non-ASCII characters from the distrito column
   # closest_missions['distrito'] = closest_missions['distrito'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # Save the cleaned data
    #closest_missions.to_csv(output_file, index=False)
#Apply the function to "C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv" and save the output to "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv"
clean_city_names("C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv")

#Now, generate a key to merge closest_missions_cleaned with survey data later.
#The key should be the cleaned city name + department code
#Hence, first geberate a function such that If departamento has only one digit, add a leading zero
def add_leading_zero(x):
    """
    Add a leading zero to single-digit department codes.
    
    Args:
        x (int or str): The department code.
    
    Returns:
        str: The department code with a leading zero if necessary.
    """
    # Convert to string
    x = str(x)
    # Check if the length is 1
    if len(x) == 1:
        return f"0{x}"
    return x

#Now, apply the function to the department_code column of "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv"
closest_missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv", encoding='latin1')
closest_missions['department_code'] = closest_missions['department_code'].apply(add_leading_zero)

#Needs a function to generate the key
#Finally, generate the key
def generate_key(closest_missions):
    """
    Generate a key for merging the closest missions data with survey data.
    
    Args:
        closest_missions (pd.DataFrame): The closest missions data.
    
    Returns:
        pd.DataFrame: The closest missions data with a key column.
    """
    # Generate the key
    closest_missions['key'] = closest_missions['distrito'] + closest_missions['department_code']
    return closest_missions

#Save the data
generate_key(closest_missions).to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv", index=False, encoding='latin1')

Closest missions have been calculated and saved!


In [168]:
#Define a function to read the survey data
def process_survey_data(input_file, output_file):
    # Read the .dta file using pyreadstat
    survey_data, metadata = pyreadstat.read_dta(input_file)
    # Replace the codes in 'distrito' with their corresponding labels
    distrito_labels = metadata.value_labels[metadata.variable_to_label['distrito']]
    survey_data['distrito'] = survey_data['distrito'].map(distrito_labels)
    #Make departamento variable string
    print("Before leading zero fix:", survey_data['departamento'].unique())
    survey_data['departamento'] = survey_data['departamento'].astype(str)
    # If departamento has only one digit, add a leading zero
    survey_data['departamento'] = survey_data['departamento'].apply(
    lambda x: x.zfill(2) if x.isdigit() else x
    )
    print("After leading zero fix:", survey_data['departamento'].unique())
    # Remove accents and non-ASCII characters
    survey_data['distrito'] = survey_data['distrito'].apply(lambda x: unidecode(x))
    # Save the cleaned data
    survey_data.to_csv(output_file, index=False, encoding='latin1')
    # Return the cleaned data
    return survey_data

#Apply the function to the survey_data
survey_data = process_survey_data("C:/Users/HP/Desktop/Felipe/Encuesta/survey_final.dta", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv")

#Save the data
survey_data_cleaned = pd.read_csv(
    "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", 
    encoding='latin1'
)

print(survey_data_cleaned.head())

Before leading zero fix: [10  8  4  6  7]
After leading zero fix: ['10' '08' '04' '06' '07']
   nro_cuestionario  tipo_formulario  departamento           distrito  \
0              1241                1            10       hernandarias   
1              1265                2             8  san juan bautista   
2              1042                1             4               Numi   
3              1089                2             8         san miguel   
4              1127                2             8         san miguel   

   localidad          s          o  altitud  gps_nro  fecha_entrevista_dia  \
0        564 -25.410032 -54.645100    227.0       21                    27   
1        550 -26.674168 -57.145817    136.0       21                    23   
2        517 -25.954933 -56.329567    139.0       12                    18   
3        531 -26.536833 -57.043083    124.0       21                    17   
4        507 -26.532932 -57.042450    132.0       22                    17   


In [172]:
# Define a function to process survey data
def process_survey_data(input_file, output_file):
    # Read the .dta file using pyreadstat
    survey_data, metadata = pyreadstat.read_dta(input_file)
    
    # Replace the codes in 'distrito' with their corresponding labels
    distrito_labels = metadata.value_labels[metadata.variable_to_label['distrito']]
    survey_data['distrito'] = survey_data['distrito'].map(distrito_labels)
    
    # Debug: Inspect 'departamento' before processing
    print("Before processing 'departamento':", survey_data['departamento'].unique())
    
    # Ensure 'departamento' is a string and strip whitespace
    survey_data['departamento'] = survey_data['departamento'].astype(str).str.strip()
    
    # Add a leading zero if 'departamento' has only one digit
    survey_data['departamento'] = survey_data['departamento'].apply(
    lambda x: x.zfill(2) if x.isdigit() else x
    )
    
    # Debug: Inspect 'departamento' after processing
    print("After processing 'departamento':", survey_data['departamento'].unique())
    
    # Remove accents and non-ASCII characters from 'distrito'
    survey_data['distrito'] = survey_data['distrito'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
    
    # Save the cleaned data
    survey_data.to_csv(output_file, index=False, encoding='latin1')
    
    # Return the cleaned DataFrame
    return survey_data

# Apply the function to the survey data
survey_data = process_survey_data(
    "C:/Users/HP/Desktop/Felipe/Encuesta/survey_final.dta", 
    "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv"
)

# Load the saved data to confirm it's cleaned
#survey_data_cleaned = pd.read_excel("survey_data_cleaned.xlsx")
#print(survey_data_cleaned.head())


Before processing 'departamento': [10  8  4  6  7]
After processing 'departamento': ['10' '08' '04' '06' '07']


In [170]:

#Define a function to generate a key for the survey data so I can merge later with the closest_missions data
def generate_key(survey_data):
    #Create a new column 'key' that combines 'distrito' and dptocode
    survey_data['key'] = survey_data['distrito'] + survey_data['departamento'].astype(str)

#Apply the function to the survey data
survey_data = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", encoding='latin1')
generate_key(survey_data)
#Save the data
survey_data.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", index=False)

In [171]:
import pandas as pd

def merge_survey_and_missions(survey_file, missions_file, key_column):
    """
    Loads survey data and closest missions data, then merges them on the specified key column.

    Parameters:
    - survey_file (str): Path to the survey data CSV file.
    - missions_file (str): Path to the closest missions data CSV file.
    - key_column (str): Column name to merge on.
    - encoding (str): Encoding of the CSV files (default is 'latin1').

    Returns:
    - pd.DataFrame: Merged DataFrame.
    """        
    # Load the survey data
    survey_data = pd.read_csv(survey_file)
    print(f"Loaded survey data with {len(survey_data)} rows.")

    # Load the closest missions data
    closest_missions = pd.read_csv(missions_file)
    print(f"Loaded closest missions data with {len(closest_missions)} rows.")

    # Merge the two data sets on the key column
    merged_data = pd.merge(survey_data, closest_missions, on=key_column, how='left')
    print(f"Merged data contains {len(merged_data)} rows.")
    # Save the merged data
    merged_data.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_with_missions.csv", index=False)

survey_file = "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv"
missions_file = "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv"
#Apply the function to the survey data and closest_missions data
merge_survey_and_missions("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/survey_data_cleaned.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/closest_missions_cleaned.csv", "key")


Loaded survey data with 560 rows.
Loaded closest missions data with 91 rows.
Merged data contains 560 rows.
